[View in Colaboratory](https://colab.research.google.com/github/malinenimaurya/Deep-Learning/blob/master/mnist_multi_layer.ipynb)

In [0]:
#Added dropout to address overfitting, used random weight initialisation
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


In [0]:
# model (added multiple hidden layers )
pkeep = tf.placeholder(tf.float32)
X = tf.placeholder(tf.float32, [None, 784])
W1 = tf.Variable(tf.truncated_normal([784, 200], stddev=0.1)) #random initialisations of weights
b1 = tf.Variable(tf.ones([200])/10)                           #small positive initialisation for bias
W2 = tf.Variable(tf.truncated_normal([200,100], stddev=0.1))
b2 = tf.Variable(tf.ones([100])/10)
W3 = tf.Variable(tf.truncated_normal([100,50], stddev=0.1))
b3 = tf.Variable(tf.ones([50])/10)
W4 = tf.Variable(tf.truncated_normal([50,25], stddev=0.1))
b4 = tf.Variable(tf.zeros([25]))
W5 = tf.Variable(tf.truncated_normal([25,10], stddev=0.1))
b5 = tf.Variable(tf.zeros([10]))
Y1 = tf.nn.relu(tf.matmul(X,W1)+b1)                           #can use sigmoid,tanh or relu
Y1d = tf.nn.dropout(Y1, pkeep)                                #Added dropout
Y2 = tf.nn.relu(tf.matmul(Y1d, W2) + b2)
Y2d = tf.nn.dropout(Y2, pkeep)
Y3 = tf.nn.relu(tf.matmul(Y2d, W3) + b3)
Y3d = tf.nn.dropout(Y3, pkeep)
Y4 = tf.nn.relu(tf.matmul(Y3d, W4) + b4)
Ylogits = tf.matmul(Y4,W5) + b5
Y = tf.nn.softmax(Ylogits)
# placeholder for correct labels
Y_ = tf.placeholder(tf.float32, [None, 10])

# loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_) # handled nan(log0) for cross entropy, different optimisers
cross_entropy = tf.reduce_mean(cross_entropy)*100

# % of correct answers found in batch
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [0]:
optimizer = tf.train.AdamOptimizer(0.003)         #can use RMSprop, Adadelta, gradientdescent etc
train_step = optimizer.minimize(cross_entropy)

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
accuracy_history = []
for i in range(1000):
    # load batch of images and correct answers
    batch_X, batch_Y = mnist.train.next_batch(100)
    train_data={X: batch_X, Y_: batch_Y, pkeep : 0.7}

    # train
    sess.run(train_step, feed_dict=train_data)
    if i%50 == 0:
      a,c = sess.run([accuracy, cross_entropy], feed_dict=train_data)
      accuracy_history.append(a)
      print("iteration:"+ str(i) + ", accuracy:" + str(a) +", cross_entropy:"+ str(c))

In [0]:
plt.plot(range(len(accuracy_history)), accuracy_history)
plt.xlabel('Iterations ')
plt.ylabel('Accuracy_History ')
plt.title("Accuracy per 50 iterations Graph")
plt.show()

In [0]:
a,c = sess.run([accuracy, cross_entropy], feed_dict=train_data)
print(a,c)

In [0]:

test_data={X: mnist.test.images, Y_: mnist.test.labels, pkeep : 1}
a,c = sess.run([accuracy, cross_entropy], feed_dict=test_data)

In [0]:
print(a,c)